In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from pathlib import Path
from glob import glob

path_start = Path("/content/drive/MyDrive/Nishika/Mansion_2023_spring")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv(path_start/"Inport_data/L01-2023P-48-01.0a/L01-2023P-2K.csv", encoding="shift_jis")
display(df.columns)
display(df.shape)
df.head()

Index(['経度', '緯度', '所在地コード', '用途', '連番', '年次', '前年所在地コード', '前年用途', '前年連番',
       '市区町村名',
       ...
       '属性移動Ｈ２６', '属性移動Ｈ２７', '属性移動Ｈ２８', '属性移動Ｈ２９', '属性移動Ｈ３０', '属性移動Ｈ３１',
       '属性移動Ｒ２', '属性移動Ｒ３', '属性移動Ｒ４', '属性移動Ｒ５'],
      dtype='object', length=128)

(25993, 128)

,経度,緯度,所在地コード,用途,連番,年次,前年所在地コード,前年用途,前年連番,市区町村名,...,属性移動Ｈ２６,属性移動Ｈ２７,属性移動Ｈ２８,属性移動Ｈ２９,属性移動Ｈ３０,属性移動Ｈ３１,属性移動Ｒ２,属性移動Ｒ３,属性移動Ｒ４,属性移動Ｒ５
0,508730.562,154999.616,1101,0,1,2023,1101,0,1,札幌中央,...,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000
1,508821.829,154950.512,1101,0,2,2023,1101,0,2,札幌中央,...,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000
2,508840.405,154969.668,1101,0,3,2023,1101,0,3,札幌中央,...,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000
3,508857.995,154930.896,1101,0,4,2023,1101,0,4,札幌中央,...,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000010000000,10000000000000,10000000000000,10000000000000,10000000000000
4,508735.091,154958.029,1101,0,5,2023,1101,0,5,札幌中央,...,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000,10000000000000


In [ ]:
a = df[["所在地コード", "Ｒ５価格"]]
b = a.groupby("所在地コード", as_index=False).mean()
b = b.rename(columns={"所在地コード":"市区町村コード"})
b["Ｒ５価格"] = (b["Ｒ５価格"] - b["Ｒ５価格"].mean())/ b["Ｒ５価格"].std()
b

,市区町村コード,Ｒ５価格
0,1101,1.512128
1,1102,0.192108
2,1103,0.038747
3,1104,0.040535
4,1105,0.119925
...,...,...
1524,47328,0.027368
1525,47329,-0.048628
1526,47348,-0.067540
1527,47350,0.015221


In [ ]:
df_train = pd.read_csv(path_start/"input/df_train.csv")
df_test = pd.read_csv(path_start/"input/test.csv")
df_con = pd.concat([df_train, df_test])
df_up = pd.merge(df_con, b, on="市区町村コード", how="left")
df_up[["ID", "Ｒ５価格"]].head()

,ID,Ｒ５価格
0,8067540,-0.164178
1,8027265,-0.169847
2,8061526,-0.095938
3,8086147,-0.095938
4,8049498,0.019509


In [ ]:
display(df_up.shape)
df_up["Ｒ５価格"] = df_up["Ｒ５価格"].fillna(0)
display(df_up.isna().sum())

(784713, 30)

Unnamed: 0           0
ID                   0
種類                   0
地域              784713
市区町村コード              0
都道府県名                0
市区町村名                0
地区名                672
最寄駅：名称            2800
最寄駅：距離（分）            0
間取り              29520
面積（㎡）                0
土地の形状           784713
間口              784713
延床面積（㎡）         784713
建築年              22956
建物の構造            22552
用途               86921
今後の利用目的         368468
前面道路：方位         784713
前面道路：種類         784713
前面道路：幅員（ｍ）      784713
都市計画             20698
建ぺい率（％）          25357
容積率（％）           25357
取引時点                 0
改装               77297
取引の事情等          765272
取引価格（総額）_log     19301
Ｒ５価格                 0
dtype: int64

In [ ]:
FileName = "df_国土数値"
df_up[["ID", "Ｒ５価格"]].to_csv(path_start/"feature_value/{}.csv".format(FileName), index = False)
pd.read_csv(path_start/"feature_value/{}.csv".format(FileName))

,ID,Ｒ５価格
0,8067540,-0.164178
1,8027265,-0.169847
2,8061526,-0.095938
3,8086147,-0.095938
4,8049498,0.019509
...,...,...
784708,47006020,-0.037466
784709,47006229,-0.037466
784710,47006331,-0.037466
784711,47006332,-0.037466
